# <font color='green'>Sistema web inteligente para recomendação de manutenção preventiva em equipamentos industriais</font>

O projeto tem como objetivo desenvolver uma aplicação web para um sistema de recomendação de manutenção preventiva em equipamentos industriais. <br>Para isso, será construído um modelo de Machine Learning capaz de prever a necessidade de manutenção com base em dados coletados por sensores IoT.

Como as classes apresentam um leve desbalanceamento, serão avaliadas cinco estratégias de balanceamento, a fim de identificar a mais adequada, acompanhada da devida justificativa. <br>Os dados utilizados são fictícios e simulam o funcionamento de um equipamento industrial monitorado por cinco sensores.

In [1]:
# Imports
import joblib
import sklearn
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE

## Carregando os Dados e Verificando Proporção de Classe

In [2]:
# Carrega o dataset
df = pd.read_csv('dataset-ml.csv')

In [3]:
# Shape
df.shape

(10000, 6)

In [4]:
# Amostra
df.head()

,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,manutencao
0,0.248937,92.270831,100.291774,67.519084,7499,0
1,0.889077,69.041692,96.070934,70.610098,600,0
2,0.565632,66.372879,93.651855,31.751346,6919,0
3,0.845819,81.890151,101.915938,46.480098,4032,1
4,2.156719,60.117687,97.461790,50.206367,8036,0


In [5]:
# Info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   sensor_1    10000 non-null  float64
 1   sensor_2    10000 non-null  float64
 2   sensor_3    10000 non-null  float64
 3   sensor_4    10000 non-null  float64
 4   sensor_5    10000 non-null  int64  
 5   manutencao  10000 non-null  int64  
dtypes: float64(4), int64(2)
memory usage: 468.9 KB


## Proporção de classe

In [6]:
df.manutencao.value_counts()

manutencao
1    5517
0    4483
Name: count, dtype: int64

- 1 é a classe positiva (foi necessário manutenção na máquina)
- 0 é a classe negativa (não foi necessário manutenção na máquina)

Observa-se um leve desbalanceamento entre as classes: dos 10 mil registros, 55,17% (5.517) indicam a necessidade de manutenção.

### Desbalanceamento de Classes
O desbalanceamento de classes ocorre quando a distribuição das classes em um conjunto de dados é desigual, com uma classe apresentando significativamente mais exemplos que as demais.

Esse fenômeno é comum em cenários de Business Analytics, especialmente em tarefas de classificação, como detecção de fraudes, previsão de churn ou identificação de leads qualificados.

Quando um modelo é treinado em dados desbalanceados, tende a favorecer a classe majoritária, comprometendo a capacidade de identificar corretamente a classe minoritária — que, em muitos casos, é justamente a mais relevante.

#### Como verificar o desbalanceamento de classes?
Durante a análise exploratória, a distribuição das classes pode ser verificada por meio de gráficos (como barras ou histogramas) e cálculo da proporção de ocorrências por classe.

Por exemplo, em um conjunto de dados de churn, se 90% dos registros correspondem a clientes que permaneceram e apenas 10% a clientes que cancelaram o serviço, há um desbalanceamento evidente.

#### Quando fazer o tratamento do desbalanceamento de classes?
O tratamento é recomendado especialmente nos seguintes casos:

- **A classe minoritária tem relevância estratégica**
<br>Quando representa eventos críticos — como fraudes —, erros de classificação podem gerar impactos significativos no negócio.

- **O modelo apresenta viés para a classe majoritária**
<br>Se a acurácia global é alta, mas o desempenho na classe minoritária é baixo (ex.: precision, recall ou F1-score), o desbalanceamento pode estar comprometendo a efetividade do modelo.



## Preparação dos Dados

In [7]:
# Separar variáveis explicativas (X) e variável alvo (y)
X = df.drop('manutencao', axis = 1)
y = df['manutencao']

In [8]:
# Dividir os dados em conjunto de treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

## Opção 1 - Ajuste de Pesos no Modelo (Sem Reamostragem)

[https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

O parâmetro class_weight='balanced' é uma forma eficiente e automática de lidar com classes desbalanceadas diretamente no modelo, tornando-o mais justo e eficaz, sem a necessidade de alterar os dados originais.

In [9]:
# Padronizar os dados 
scaler_v1 = StandardScaler()
X_train_scaled = scaler_v1.fit_transform(X_train)
X_test_scaled = scaler_v1.transform(X_test)

In [10]:
# Instanciar o modelo com ajuste de pesos
modelo_v1 = RandomForestClassifier(n_estimators = 100, 
                                   random_state = 42, 
                                   class_weight = 'balanced')

In [11]:
%%time
modelo_v1.fit(X_train_scaled, y_train)

CPU times: user 974 ms, sys: 6.45 ms, total: 980 ms
Wall time: 980 ms


RandomForestClassifier(class_weight='balanced', random_state=42)

In [12]:
# Fazer previsões no conjunto de teste
y_pred = modelo_v1.predict(X_test_scaled)
y_pred_proba = modelo_v1.predict_proba(X_test_scaled)[:, 1]

In [13]:
# Avaliação do modelo
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"\nAcurácia: {accuracy * 100:.2f}%")
print(f"AUC-ROC: {roc_auc * 100:.2f}%")
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))


Acurácia: 97.95%
AUC-ROC: 98.87%

Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.97      0.98      0.98       876
           1       0.99      0.98      0.98      1124

    accuracy                           0.98      2000
   macro avg       0.98      0.98      0.98      2000
weighted avg       0.98      0.98      0.98      2000



In [14]:
# Salva modelo e padronizador em disco
model_file = 'modelos/modelo_v1.pkl'
scaler_file = 'padronizadores/scaler_v1.pkl'

joblib.dump(modelo_v1, model_file)
joblib.dump(scaler_v1, scaler_file)

print(f"Modelo salvo em {model_file}")
print(f"Scaler salvo em {scaler_file}")

Modelo salvo em modelos/modelo_v1.pkl
Scaler salvo em padronizadores/scaler_v1.pkl


## Opção 2 - Subamostragem da Classe Majoritária (Undersampling)

Neste cenário, opta-se por utilizar a técnica de undersampling para lidar com o desbalanceamento das classes ao aplicar o algoritmo Random Forest. <br>Em vez de ajustar os pesos das classes por meio do parâmetro class_weight='balanced', essa abordagem reduz a quantidade de amostras da classe majoritária, equilibrando o conjunto de dados de forma manual.

Essa estratégia visa evitar que o modelo favoreça a classe mais representada, permitindo que a Random Forest aprenda de maneira mais equitativa a distinguir ambas as classes. <br>Apesar de implicar na perda de parte das informações da classe majoritária, o undersampling pode ser eficaz quando o volume de dados é suficientemente grande, reduzindo viés e melhorando a capacidade preditiva sobre a classe minoritária.

In [15]:
# Proporção de classe
df.manutencao.value_counts()

manutencao
1    5517
0    4483
Name: count, dtype: int64

In [16]:
# Concatenar X_train e y_train para facilitar a reamostragem
train_data = pd.concat([X_train, y_train], axis = 1)

In [17]:
# Separar a classe majoritária e minoritária do conjunto de treino
df_majoritaria = train_data[train_data.manutencao == 1]
df_minoritaria = train_data[train_data.manutencao == 0]

Lembre-se que aplicamos reamostragem nos dados de treino!

In [18]:
len(df_majoritaria)

4393

In [19]:
len(df_minoritaria)

3607

In [20]:
# Aplicar subamostragem da classe majoritária no conjunto de treino
df_majoritaria_subamostrada = resample(df_majoritaria,
                                       replace = False,    
                                       n_samples = len(df_minoritaria),  # Igualar o número da classe minoritária
                                       random_state = 42)

In [21]:
# Combinar as classes minoritária e majoritária subamostrada
train_data_balanceado = pd.concat([df_majoritaria_subamostrada, df_minoritaria])

In [22]:
# Separar novamente em X_train e y_train balanceados
X_train_balanceado = train_data_balanceado.drop('manutencao', axis = 1)
y_train_balanceado = train_data_balanceado['manutencao']

In [23]:
# Verificar o balanceamento no conjunto de treino
print(y_train_balanceado.value_counts())

manutencao
1    3607
0    3607
Name: count, dtype: int64


In [24]:
# Padronizar os dados
scaler_v2 = StandardScaler()
X_train_scaled = scaler_v2.fit_transform(X_train_balanceado)
X_test_scaled = scaler_v2.transform(X_test)

In [25]:
# Instanciar e treinar o modelo
modelo_v2 = RandomForestClassifier(n_estimators = 100, random_state = 42)

In [26]:
%%time
modelo_v2.fit(X_train_scaled, y_train_balanceado)

CPU times: user 851 ms, sys: 7.69 ms, total: 859 ms
Wall time: 858 ms


RandomForestClassifier(random_state=42)

In [27]:
# Avaliar o modelo no conjunto de teste 
y_pred = modelo_v2.predict(X_test_scaled)
y_pred_proba = modelo_v2.predict_proba(X_test_scaled)[:, 1]

In [28]:
# Avaliação do modelo
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"\nAcurácia: {accuracy * 100:.2f}%")
print(f"AUC-ROC: {roc_auc * 100:.2f}%")
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))


Acurácia: 97.30%
AUC-ROC: 98.23%

Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.95      0.99      0.97       876
           1       0.99      0.96      0.98      1124

    accuracy                           0.97      2000
   macro avg       0.97      0.97      0.97      2000
weighted avg       0.97      0.97      0.97      2000



In [29]:
# Salva modelo e padronizador em disco
model_file = 'modelos/modelo_v2.pkl'
scaler_file = 'padronizadores/scaler_v2.pkl'

joblib.dump(modelo_v2, model_file)
joblib.dump(scaler_v2, scaler_file)

print(f"Modelo salvo em {model_file}")
print(f"Scaler salvo em {scaler_file}")

Modelo salvo em modelos/modelo_v2.pkl
Scaler salvo em padronizadores/scaler_v2.pkl


## Opção 3 - Superamostragem da Classe Minoritária (Oversampling)

Aqui utiliza-se a estratégia de oversampling para lidar com o desbalanceamento das classes ao aplicar o algoritmo Random Forest, sem recorrer ao parâmetro class_weight='balanced'.<br> Essa abordagem consiste em replicar aleatoriamente amostras da classe minoritária até que o número de instâncias se iguale ou se aproxime ao da classe majoritária, equilibrando o conjunto de dados antes do treinamento.

Ao aumentar a presença da classe minoritária, o modelo é exposto de forma mais equilibrada às diferentes classes, o que pode melhorar sua capacidade de identificar corretamente padrões relacionados à classe menos representada. <br>Essa técnica preserva todas as informações da classe majoritária e é especialmente útil quando há dados limitados da classe minoritária, sem introduzir novas instâncias artificiais.

Embora o oversampling por replicação possa aumentar o risco de overfitting, principalmente se a base de dados for pequena, ele continua sendo uma alternativa simples e eficaz para melhorar o desempenho do modelo em cenários desbalanceados.

In [30]:
# Concatenar X_train e y_train para facilitar a reamostragem
train_data = pd.concat([X_train, y_train], axis = 1)

In [31]:
# Separar a classe majoritária e minoritária do conjunto de treino
df_majoritaria = train_data[train_data.manutencao == 1]
df_minoritaria = train_data[train_data.manutencao == 0]

Lembre-se que aplicamos reamostragem nos dados de treino!

In [32]:
len(df_majoritaria)

4393

In [33]:
len(df_minoritaria)

3607

In [34]:
# Superamostragem da classe minoritária no conjunto de treino
df_minoritaria_superamostrada = resample(df_minoritaria,
                                         replace = True,     
                                         n_samples = len(df_majoritaria), # Igualar ao número da classe majoritária
                                         random_state = 42)

In [35]:
# Combinar as classes majoritária e minoritária superamostrada
train_data_balanceado = pd.concat([df_majoritaria, df_minoritaria_superamostrada])

In [36]:
# Separar novamente em X_train e y_train balanceados
X_train_balanceado = train_data_balanceado.drop('manutencao', axis = 1)
y_train_balanceado = train_data_balanceado['manutencao']

In [37]:
# Verificar o balanceamento no conjunto de treino
print(y_train_balanceado.value_counts())

manutencao
1    4393
0    4393
Name: count, dtype: int64


In [38]:
# Padronizar os dados
scaler_v3 = StandardScaler()
X_train_scaled = scaler_v3.fit_transform(X_train_balanceado)
X_test_scaled = scaler_v3.transform(X_test)

In [39]:
# Criar o modelo
modelo_v3 = RandomForestClassifier(n_estimators = 100, random_state = 42)

In [40]:
%%time
modelo_v3.fit(X_train_scaled, y_train_balanceado)

CPU times: user 972 ms, sys: 3.51 ms, total: 976 ms
Wall time: 975 ms


RandomForestClassifier(random_state=42)

In [41]:
# Avaliar o modelo no conjunto de teste 
y_pred = modelo_v3.predict(X_test_scaled)
y_pred_proba = modelo_v3.predict_proba(X_test_scaled)[:, 1]

In [42]:
# Avaliação do modelo
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"\nAcurácia: {accuracy * 100:.2f}%")
print(f"AUC-ROC: {roc_auc * 100:.2f}%")
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))


Acurácia: 97.65%
AUC-ROC: 98.88%

Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.97      0.97      0.97       876
           1       0.98      0.98      0.98      1124

    accuracy                           0.98      2000
   macro avg       0.98      0.98      0.98      2000
weighted avg       0.98      0.98      0.98      2000



In [43]:
# Salva modelo e padronizador em disco
model_file = 'modelos/modelo_v3.pkl'
scaler_file = 'padronizadores/scaler_v3.pkl'

joblib.dump(modelo_v3, model_file)
joblib.dump(scaler_v3, scaler_file)

print(f"Modelo salvo em {model_file}")
print(f"Scaler salvo em {scaler_file}")

Modelo salvo em modelos/modelo_v3.pkl
Scaler salvo em padronizadores/scaler_v3.pkl


## Opção 4 - Balanceamento Automático com SMOTE

SMOTE é uma técnica utilizada para gerar novas amostras da classe minoritária com base nas distâncias entre os pontos. <br>Como esse processo é sensível à escala dos dados, variáveis com magnitudes maiores (por exemplo, "sensor_5" em comparação a "sensor_6") podem ter influência desproporcional na criação das novas amostras. <br>Por esse motivo, os dados são padronizados antes da aplicação do SMOTE.<br>
Nesta etapa, o algoritmo Random Forest está sendo utilizado sem o uso do parâmetro class_weight='balanced'.

In [44]:
# Padronização
scaler_v4 = StandardScaler()
X_train_scaled = scaler_v4.fit_transform(X_train)
X_test_scaled = scaler_v4.transform(X_test)

In [45]:
# Cria o SMOTE
smote = SMOTE(random_state = 42)

In [46]:
# Treinar e aplicar SMOTE no conjunto de treino
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

In [47]:
len(X_train_smote)

8786

In [48]:
# Criar o modelo
modelo_v4 = RandomForestClassifier(n_estimators = 100, random_state = 42)

In [49]:
%%time
modelo_v4.fit(X_train_smote, y_train_smote)

CPU times: user 1.05 s, sys: 7.06 ms, total: 1.06 s
Wall time: 1.06 s


RandomForestClassifier(random_state=42)

In [50]:
# Avaliar o modelo no conjunto de teste 
y_pred = modelo_v4.predict(X_test_scaled)
y_pred_proba = modelo_v4.predict_proba(X_test_scaled)[:, 1]

In [51]:
# Avaliação do modelo
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"\nAcurácia: {accuracy * 100:.2f}%")
print(f"AUC-ROC: {roc_auc * 100:.2f}%")
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))


Acurácia: 97.95%
AUC-ROC: 98.85%

Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.97      0.98      0.98       876
           1       0.99      0.98      0.98      1124

    accuracy                           0.98      2000
   macro avg       0.98      0.98      0.98      2000
weighted avg       0.98      0.98      0.98      2000



In [52]:
# Salva modelo e padronizador em disco
model_file = 'modelos/modelo_v4.pkl'
scaler_file = 'padronizadores/scaler_v4.pkl'

joblib.dump(modelo_v4, model_file)
joblib.dump(scaler_v4, scaler_file)

print(f"Modelo salvo em {model_file}")
print(f"Scaler salvo em {scaler_file}")

Modelo salvo em modelos/modelo_v4.pkl
Scaler salvo em padronizadores/scaler_v4.pkl


## Opção 5 - Balanceamento Automático com SMOTE e Mudança de Algoritmo

Nesta abordagem, o algoritmo LightGBM é aplicado a um conjunto de dados previamente balanceado utilizando a técnica SMOTE. <br>O balanceamento é realizado antes do treinamento, com o objetivo de corrigir a desproporção entre as classes e permitir que o modelo aprenda de forma mais representativa sobre todas as categorias envolvidas.

Ao aplicar o SMOTE previamente, evita-se o uso de parâmetros internos de balanceamento, como is_unbalance ou scale_pos_weight, permitindo que o LightGBM opere sobre um conjunto de dados já ajustado. <br>Essa estratégia favorece a identificação de padrões da classe minoritária, podendo resultar em ganhos significativos de desempenho em métricas como recall, F1-score e AUC.

O uso do SMOTE aliado ao LightGBM é especialmente vantajoso em cenários com forte desbalanceamento, em que o modelo, treinado diretamente nos dados originais, tende a negligenciar a classe menos representada.

In [53]:
%%time

# Padronizar as variáveis
scaler_v5 = StandardScaler()
X_train_scaled = scaler_v5.fit_transform(X_train)
X_test_scaled = scaler_v5.transform(X_test)

# Criar o SMOTE
smote = SMOTE(random_state = 42)

# Aplicar SMOTE no conjunto de treino para lidar com o desbalanceamento
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

# Criar o modelo LightGBM
modelo_v5 = lgb.LGBMClassifier(random_state = 42)

# Treinar o modelo LightGBM
modelo_v5.fit(X_train_smote, y_train_smote)

# Avaliar o modelo no conjunto de teste original
y_pred = modelo_v5.predict(X_test_scaled)
y_pred_proba = modelo_v5.predict_proba(X_test_scaled)[:, 1]

# Avaliação do modelo
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"\nAcurácia: {accuracy * 100:.2f}%")
print(f"AUC-ROC: {roc_auc * 100:.2f}%")
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))

[LightGBM] [Info] Number of positive: 4393, number of negative: 4393
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000144 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 8786, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000

Acurácia: 91.00%
AUC-ROC: 97.11%

Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.85      0.97      0.90       876
           1       0.97      0.86      0.92      1124

    accuracy                           0.91      2000
   macro avg       0.91      0.92      0.91      2000
weighted avg       0.92      0.91      0.91      2000

CPU times: user 364 ms, sys: 4.87 ms, total: 369 ms
Wall time: 79 ms


In [54]:
# Salva modelo e padronizador em disco
model_file = 'modelos/modelo_v5.pkl'
scaler_file = 'padronizadores/scaler_v5.pkl'

joblib.dump(modelo_v5, model_file)
joblib.dump(scaler_v5, scaler_file)

print(f"Modelo salvo em {model_file}")
print(f"Scaler salvo em {scaler_file}")

Modelo salvo em modelos/modelo_v5.pkl
Scaler salvo em padronizadores/scaler_v5.pkl


## Seleção de Modelo

Versão 1 do Modelo:

- Acurácia: 97.95%
- AUC-ROC: 98.87%

Versão 2 do Modelo:

- Acurácia: 97.30%
- AUC-ROC: 98.23%

Versão 3 do Modelo:

- Acurácia: 97.65%
- AUC-ROC: 98.88%

Versão 4 do Modelo:

- Acurácia: 97.95%
- AUC-ROC: 98.85%

Versão 5 do Modelo:

SMOTE Primeiro e Padronização Depois!

- Acurácia: 96.65%
- AUC-ROC: 98.39%

Padronização Primeiro e SMOTE Depois! (Essa é a ordem ideal)

- Acurácia: 91.00%
- AUC-ROC: 97.11%

**Qual modelo poderia ser escolhido e com qual justificativa?**

### Justificativa da escolha do modelo

Um modelo de Machine Learning é construído a partir da combinação de algoritmo e dados. <br>Nesse contexto, consideramos um modelo simples aquele que atinge boa performance com o mínimo de ajustes tanto nos dados quanto no algoritmo.

Com base nesse critério, a versão 1 do modelo foi escolhida para o deploy, por apresentar o melhor equilíbrio entre os seguintes fatores:

- Capacidade de generalização
- Performance
- Simplicidade
- Interpretabilidade

Nosso objetivo é sempre encontrar o modelo que concilie esses quatro elementos de forma harmônica

## Testando o Deploy do Modelo

In [55]:
# Função para recomendar manutenção baseada em novos dados de sensores IoT
def recomendacao_manutencao(novo_dado):
    
    # Definir os nomes das colunas conforme o scaler foi ajustado
    colunas = ['sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5']
    
    # Converter os novos dados para DataFrame com os nomes de colunas corretos
    novo_dado_df = pd.DataFrame([novo_dado], columns = colunas)
    
    # Aplicar o scaler aos novos dados
    novo_dado_scaled = scaler_v1.transform(novo_dado_df)
    
    # Fazer a previsão
    predicao = modelo_v1.predict(novo_dado_scaled)
    
    if predicao == 1:
        return "Recomendação: Realizar manutenção."
    else:
        return "Recomendação: Nenhuma manutenção necessária."

In [56]:
# Exemplo de novos dados de sensores IoT 
novos_dados_1 = [0.5, 80, 102, 45, 8000]
print(recomendacao_manutencao(novos_dados_1))

Recomendação: Realizar manutenção.


In [57]:
# Exemplo de novos dados de sensores IoT 
novos_dados_2 = [0.89, 92, 96, 70, 600]
print(recomendacao_manutencao(novos_dados_2))

Recomendação: Nenhuma manutenção necessária.


**Agora pode-se fazer o deploy do modelo por meio de uma aplicação web desenvolvida com Streamlit no arquivo "web_ml_manutencao_industrial_app.py".**

# Fim